In [1]:
# https://github.com/run-llama/llama_index/blob/main/docs/docs/examples/prompts/prompts_rag.ipynb

In [2]:
# %pip install llama-index-readers-file pymupdf
# %pip install llama-index-embeddings-openai
# %pip install llama-index-embeddings-huggingface

In [3]:
import os
import openai

In [4]:
import os
os.environ['OPENAI_API_KEY']='sk-111111111111111111111111111111111111111111111111'
os.environ['OPENAI_API_BASE']='http://127.0.0.1:5000/v1'

In [5]:
import requests
model_info_url = 'http://127.0.0.1:5000/v1/internal/model/info'
resp = requests.get(model_info_url)
model = resp.json()['model_name']

print(model)

Mistral-7B-Instruct-v0.2-8.0bpw-h8-exl2-2


In [6]:
import logging
import sys

In [7]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [8]:
from llama_index.core import VectorStoreIndex
from llama_index.core import PromptTemplate
from IPython.display import Markdown, display

In [9]:
# !mkdir data
# !wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "data/llama2.pdf"

In [10]:
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader

In [11]:
loader = PyMuPDFReader()
documents = loader.load(file_path="./data/llama2.pdf")


In [12]:
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI

In [13]:
gpt35_llm = OpenAI(model="gpt-3.5-turbo")
gpt4_llm = OpenAI(model="gpt-4")

In [14]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [15]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


In [16]:
test_emeds = embed_model.get_text_embedding("Hello World!")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
len(test_emeds)

384

In [18]:
from llama_index.core import Settings

In [19]:
Settings.embed_model = embed_model

In [20]:
index = VectorStoreIndex.from_documents(documents)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
query_str = "What are the potential risks associated with the use of Llama 2 as mentioned in the context?"

In [22]:
query_engine = index.as_query_engine(similarity_top_k=2, llm=gpt35_llm)

In [23]:
vector_retriever = index.as_retriever(similarity_top_k=2)

In [24]:
response = query_engine.query(query_str)
print(str(response))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:5000/v1/chat/completions "HTTP/1.1 200 OK"
The potential risks associated with the use of Llama 2, as mentioned in the context, include the potential for biases inherited from the training data, such as underrepresentation of certain demographic groups, which could result in biased model generations. Additionally, the models carry risks related to safety violations, as mentioned in Section 4.4, which includes human evaluations of safety violations across adversarial prompts. It is important to note that these safety evaluations are subject to inherent biases due to limitations of the prompt set, subjectivity of the review guidelines, and subjectivity of individual raters. Before deploying any applications of Llama 2-Chat, developers should perform safety testing and tuning tailored to their specific applications of the model.


In [25]:
# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))

In [26]:
prompts_dict = query_engine.get_prompts()

In [27]:
display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>